In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
def read_chunks(file, cols=None, city=None, chunk_size=500000):
    df = pd.read_json(path_or_buf=file, chunksize=chunk_size, lines=True)

    chunk_list = []
    for chunk in tqdm(df, desc=file):
        if city:
            chunk = chunk[chunk['city'] == city]
        if cols is None:
            chunk_list.append(chunk)
        else:
            chunk_list.append(chunk[cols])
    
    return pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

In [3]:
city_of_interest = 'Indianapolis'

business_data = read_chunks('../yelp_dataset/yelp_academic_dataset_business.json', city=city_of_interest)
business_ids = business_data['business_id'].unique()

df = read_chunks('../yelp_dataset/yelp_academic_dataset_checkin.json')
df = df[df['business_id'].isin(business_ids)]

../yelp_dataset/yelp_academic_dataset_business.json: 0it [00:00, ?it/s]

../yelp_dataset/yelp_academic_dataset_checkin.json: 0it [00:00, ?it/s]

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6915 entries, 22 to 131863
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  6915 non-null   object
 1   date         6915 non-null   object
dtypes: object(2)
memory usage: 162.1+ KB


In [5]:
df.head()

business_id                                               date
22   --a_r_w1HTsOY-fagPeNKg  2012-04-07 16:13:35, 2012-04-28 20:40:59, 2012...
26   --hF_3v1JmU9nlu4zfXJ8Q  2021-09-15 16:08:07, 2021-09-30 17:10:48, 2021...
27   --lqIzK-ZVTtgwiQM63XgQ  2011-07-29 16:37:02, 2011-10-24 16:00:29, 2012...
60   -0gWtMKg8_iV6vC5wRFDiA  2011-10-18 20:41:06, 2011-12-10 23:08:40, 2012...
100  -1q9ktAQXpgFSNrst7tzag  2012-03-02 00:15:00, 2012-03-22 16:19:15, 2012...

In [6]:
df.duplicated().sum()

0

In [7]:
df['date'] = df['date'].str.split(', ')
df = df.explode('date')